In [1]:
from compact_rules import *

In [2]:
import pandas as pd

/home/agent/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/agent/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
df = pd.read_csv('wiki_ADV.csv')

In [4]:
df.head()

,CUR_WORD,TARGET_WORD
0,абактериально,абактериальный
1,абдоминально,абдоминальный
2,аберрантно,аберрантный
3,аберрационно,аберрационный
4,аберрометрически,аберрометрический


In [5]:
df.shape

(6747, 2)

In [6]:
advs_test = []

In [7]:
test = Derivation()

In [8]:
pos = 0
neg = 0

In [9]:
pos_examples = []
neg_examples = []

In [10]:
for i in range(df.shape[0]):
    s = df[i:i+1].values[0]
    
    w_a = s[0].lower()
    w_b = s[1].lower()
    
    advs_test.append(w_a.replace('ё', 'е'))

    correct_generated = False
    
    pos_a = 'adv'
    
    ans = []
    
    for pos_b in {'adj', 'adv', 'noun', 'verb', 'num'}:
        for rule in rules_all[pos_a]:
            if rule.pos_b == pos_b:
                ans += test.apply_pattern(POS(w_b, pos_b), rule)
    
    for elem in ans:
        if elem.graphics.lower().replace('ё', 'е') == w_a.replace('ё', 'е'):
            correct_generated = True
    if correct_generated:
        pos += 1
        pos_examples.append(w_a)
    else:
        neg += 1
        neg_examples.append(w_a)

In [11]:
print(pos, neg)

6217 530


In [12]:
for neg_ex in neg_examples:
    print(neg_ex)

босиком
броско
в-восьмых
в-девятых
в-десятых
в-пятых
в-седьмых
в-четвёртых
в-шестых
вблизи
вверху
ввечеру
вволю
ввосьмером
вгорячах
вдалеке
вдали
вдвоём
вдвойне
вдевятером
вдесятером
вдобавок
вдоволь
вдогонку
вдолбёжку
вдосыть
вдребезги
верхами
веско
вживе
взаймы
взаперти
взапивку
взаправду
взапуски
взахлёб
взрыд
вкоротке
вкось
вкратце
вкривь
влеве
вмале
вместе
вмоготу
внаброс
внаброску
внаймы
внакидку
внакладе
внакладку
внаклонку
внакрой
внапашку
внатяжку
внахлёст
внахлёстку
вначале
внизу
внове
вновь
внутри
внутрь
во-вторых
вовек
вовеки
вовне
вовнутрь
вовремя
вовсе
воистину
вокруг
волей
вообще
вослед
впервые
впервь
вперебежку
вперебивку
вперебой
вперевёртку
вперегонки
вперегонку
впереди
вперемежку
впеременку
вперемешку
вперерыв
вперехлёст
вповал
вполглаза
вполголоса
вполдерева
вполнакала
вполне
вполнеба
вполноги
вполоборота
вполоткрыта
вполпьяна
вполсилы
вполслуха
вполсыта
вполуоборот
вполуха
вполцены
вполшага
впоперечь
впоследнее
впоследние
впоследствии
вприглядку
впридачу
вприкуску


In [13]:
print(pos / (neg + pos))

0.9214465688454128


In [14]:
from lark import Lark
from lark.tree import pydot__tree_to_png    # Just a neat utility function

parser = Lark(r"""
    value: SIGNED_NUMBER
         | and
         | or
    or : "[" [value] ("|" value)* "]"
    and : [value] ("&" and)* ("&" value)

    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

    """, start='value')

In [104]:
text = "[[1 & 2 | 3 & 4] & 5 | 6 & 7 ]"
#text = "[[1 & 2 | 3 | 4 | 5 & 6 & 7 | 8 & 9 | 10] & 11 | 12]"

tree = parser.parse(text)

In [152]:
def opers_to_digits(text):
    opers = dict()
    text_short = ''
    round_brackets = 0
    cur_oper = ''
    for c in text:
        if c == '(':
            round_brackets += 1
        if round_brackets == 0:
            text_short += c
        else:
            cur_oper += c
        if c == ')':
            round_brackets -= 1
            if round_brackets == 0:               
                oper_str = cur_oper[1:-1]  # "try plt"
                sep_ind = oper_str.find(',')
                if sep_ind == -1:
                    args = "args={}"
                    mode_and_oper = oper_str
                else:
                    args = "args=" + oper_str[sep_ind + 1:].strip()
                    mode_and_oper = oper_str[:sep_ind]
                mode_and_oper = mode_and_oper.split()
                if len(mode_and_oper) > 1:
                    mode = mode_and_oper[0]
                    oper = mode_and_oper[1]
                else:
                    mode = 'do'
                    oper = mode_and_oper[0]
                opers[len(opers)] = oper + "(word=word, mode='" + mode + "', " + args + ")"
                cur_oper = ''
                text_short += str(len(opers)-1)
    return text_short, opers
    

In [175]:
text_full = "[(onlysfx, {'я', 'а', 'ь'}) & (delsfx, {'я', 'а', 'ь'}) | (excsfx, {'я', 'а', 'ь'}) & (replsfx, {('ей', 'ь')})] & (try plt) & (opt delvowel) & (addsfx, {'иный'})"

In [176]:
text_short, opers = opers_to_digits(text_full)

In [177]:
opers_to_digits(text_full)

('[0 & 1 | 2 & 3] & 4 & 5 & 6',
 {0: "onlysfx(word=word, mode='do', args={'я', 'а', 'ь'})",
  1: "delsfx(word=word, mode='do', args={'я', 'а', 'ь'})",
  2: "excsfx(word=word, mode='do', args={'я', 'а', 'ь'})",
  3: "replsfx(word=word, mode='do', args={('ей', 'ь')})",
  4: "plt(word=word, mode='try', args={})",
  5: "delvowel(word=word, mode='opt', args={})",
  6: "addsfx(word=word, mode='do', args={'иный'})"})

In [178]:
opers

{0: "onlysfx(word=word, mode='do', args={'я', 'а', 'ь'})",
 1: "delsfx(word=word, mode='do', args={'я', 'а', 'ь'})",
 2: "excsfx(word=word, mode='do', args={'я', 'а', 'ь'})",
 3: "replsfx(word=word, mode='do', args={('ей', 'ь')})",
 4: "plt(word=word, mode='try', args={})",
 5: "delvowel(word=word, mode='opt', args={})",
 6: "addsfx(word=word, mode='do', args={'иный'})"}

In [179]:
new_opers = dict()

In [180]:
tree = parser.parse(text_short)

In [181]:
print(tree)

Tree(value, [Tree(and, [Tree(value, [Tree(and, [Tree(value, [Tree(and, [Tree(value, [Tree(or, [Tree(value, [Tree(and, [Tree(value, [Token(SIGNED_NUMBER, '0')]), Tree(value, [Token(SIGNED_NUMBER, '1')])])]), Tree(value, [Tree(and, [Tree(value, [Token(SIGNED_NUMBER, '2')]), Tree(value, [Token(SIGNED_NUMBER, '3')])])])])]), Tree(value, [Token(SIGNED_NUMBER, '4')])])]), Tree(value, [Token(SIGNED_NUMBER, '5')])])]), Tree(value, [Token(SIGNED_NUMBER, '6')])])])


In [182]:
print(tree.pretty())

value
  and
    value
      and
        value
          and
            value
              or
                value
                  and
                    value	0
                    value	1
                value
                  and
                    value	2
                    value	3
            value	4
        value	5
    value	6



In [183]:
pydot__tree_to_png(tree, "syntax_tree.png")

In [184]:
from copy import deepcopy

In [185]:
tree.children[0].children[0].children[0].children[0].children[0].children[1]

Tree(value, [Token(SIGNED_NUMBER, '4')])

In [186]:
tree2 = deepcopy(tree)

In [187]:
tree2

Tree(value, [Tree(and, [Tree(value, [Tree(and, [Tree(value, [Tree(and, [Tree(value, [Tree(or, [Tree(value, [Tree(and, [Tree(value, [Token(SIGNED_NUMBER, '0')]), Tree(value, [Token(SIGNED_NUMBER, '1')])])]), Tree(value, [Tree(and, [Tree(value, [Token(SIGNED_NUMBER, '2')]), Tree(value, [Token(SIGNED_NUMBER, '3')])])])])]), Tree(value, [Token(SIGNED_NUMBER, '4')])])]), Tree(value, [Token(SIGNED_NUMBER, '5')])])]), Tree(value, [Token(SIGNED_NUMBER, '6')])])])

In [188]:
leaf = tree.children[0].children[0].children[0].children[0].children[0].children[1]

In [189]:
leaf = tree.children[0].children[0].children[0].children[0].children[0].children[0]

In [190]:
leaf

Tree(value, [Tree(or, [Tree(value, [Tree(and, [Tree(value, [Token(SIGNED_NUMBER, '0')]), Tree(value, [Token(SIGNED_NUMBER, '1')])])]), Tree(value, [Tree(and, [Tree(value, [Token(SIGNED_NUMBER, '2')]), Tree(value, [Token(SIGNED_NUMBER, '3')])])])])])

In [191]:
tree.children[0].children[1]

Tree(value, [Token(SIGNED_NUMBER, '6')])

In [202]:
def go_tree(tree):
    # лист
    code = ""
    if hasattr(tree.children[0], 'value'):
        command = opers[int(tree.children[0].value)]
        print(command)
        code += "# this is a leaf\n"
        """
        code += "new_res = []\n"
        code += "for word in cur_res:\n"
        code += "    new_res += " + oper + "(word=word, mode='" + mode + "', " + args + ")\n"
        code += "cur_res = new_res\n"
        #code += "return cur_res\n"
        """
    elif tree.children[0].data == 'or':
        code += "# this is an OR operation\n"
        """
        code += "new_res = []\n"
        code += "for child in tree.children[0].children:\n"
        code += "    new_res += " + "work_tree(child, cur_res)\n"
        """
        for child in tree.children[0].children:
            code += go_tree(child)
    elif tree.children[0].data == 'and':
        code += "# this is an AND operation\n"
        
        #code += "cur_res = work_tree(tree.children[0].children[0], cur_res)\n"
        code += go_tree(tree.children[0].children[0])
        #code += "new_res = work_tree(tree.children[0].children[1], cur_res)\n"
        code += go_tree(tree.children[0].children[1])
        
    return code

In [203]:
word = 'низка'
print(go_tree(leaf))

onlysfx(word=word, mode='do', args={'я', 'а', 'ь'})
delsfx(word=word, mode='do', args={'я', 'а', 'ь'})
excsfx(word=word, mode='do', args={'я', 'а', 'ь'})
replsfx(word=word, mode='do', args={('ей', 'ь')})
# this is an OR operation
# this is an AND operation
# this is a leaf
# this is a leaf
# this is an AND operation
# this is a leaf
# this is a leaf



In [204]:
print(go_tree(tree))

onlysfx(word=word, mode='do', args={'я', 'а', 'ь'})
delsfx(word=word, mode='do', args={'я', 'а', 'ь'})
excsfx(word=word, mode='do', args={'я', 'а', 'ь'})
replsfx(word=word, mode='do', args={('ей', 'ь')})
plt(word=word, mode='try', args={})
delvowel(word=word, mode='opt', args={})
addsfx(word=word, mode='do', args={'иный'})
# this is an AND operation
# this is an AND operation
# this is an AND operation
# this is an OR operation
# this is an AND operation
# this is a leaf
# this is a leaf
# this is an AND operation
# this is a leaf
# this is a leaf
# this is a leaf
# this is a leaf
# this is a leaf



In [206]:
def compute_tree(tree, cur_results: list()):
    # лист
    if hasattr(tree.children[0], 'value'):
        command = opers[int(tree.children[0].value)]
        print(command)
        new_results = []
        for word in cur_results:
            new_results += exec(command)
    elif tree.children[0].data == 'or':
        new_results = []
        for child in tree.children[0].children:
            new_results += compute_tree(child, cur_results)
    elif tree.children[0].data == 'and':
        new_results = compute_tree(tree.children[0].children[0], cur_results)
        new_results = compute_tree(tree.children[0].children[1], new_results)
        
    return new_results

In [207]:
compute_tree(tree, ['низка'])

onlysfx(word=word, mode='do', args={'я', 'а', 'ь'})


TypeError: onlysfx() missing 2 required positional arguments: 'pos_b' and 'pos_a'

In [34]:
class POS:
    def __init__(self, gr: str, pos: str, history=list()):
        self.pos = pos
        self.graphics = gr
        self.history = history


class Rule:
    def __init__(self, pos_b, pos_a, rules):
        """
        Пример rules = "[(onlysfx, {'я', 'а', 'ь'}) & (delsfx, {'я', 'а', 'ь'}) | (excsfx, {'я', 'а', 'ь'}) 
        & (replsfx, {('ей', 'ь')})] & (plt, 'try') & (delvowel, 'opt') & (addsfx, {'иный'})"
        
        """
        self.pos_b = pos_b
        self.pos_a = pos_a
        self.logical_form, self.full_operations = opers_to_digits(rules)
        self.syntax_tree = parser.parse(logical_form)
        self.inner_code = ""

In [35]:
paired = {'б', 'п', 'м', 'в', 'ф', 'д', 'т', 'з', 'с', 'н', 'л', 'р'}
velar = {'г', 'к', 'х'}
sh_hard = {'ш', 'ж'}
sh_soft = {'щ', 'ч'}

voiced = {'б', 'м', 'в', 'д', 'з', 'н', 'л', 'р', 'г', 'ж'}
unvoiced = {'п', 'ф', 'т', 'с', 'к', 'х', 'ш', 'ч', 'щ'}

plt_pairs = ({('к', 'ч'), ('г', 'ж'), ('х', 'ш')})

vowels = {'а', 'о', 'у', 'ы', 'и', 'э', 'я', 'ё', 'ю', 'е'}

sh = sh_hard | sh_soft

consonants = paired | velar | sh_hard | sh_soft

def delsfx(word, args, mode='do'):
    sfx_b_ = args
    possible_results = list()

    for sfx_b in sfx_b_:
        if mode == 'do':
            if word[-len(sfx_b):] == sfx_b:
                # возможно изменение -> изменяем
                w_a = word[:-len(sfx_b)]
                possible_word = w_a
                new_word = possible_word
                print(new_word)
                possible_results.append(new_word)

        elif mode == 'try':
            if word[-len(sfx_b):] == sfx_b:
                # возможно изменение -> изменяем
                w_a = word[:-len(sfx_b)]
                possible_word = w_a
                new_word = possible_word
                possible_results.append(new_word)
                print(new_word)
            else:
                # невозможно изменение -> не страшно
                w_a = word
                possible_word = w_a
                new_word = possible_word
                possible_results.append(new_word)
                print(new_word)

        elif mode == 'opt':
            if word[-len(sfx_b):] == sfx_b:
                # возможно изменение -> изменяем или не изменяем
                w_a = word[:-len(sfx_b)]
                possible_word = w_a
                new_word = possible_word
                possible_results.append(new_word)
                print(new_word)
            w_a = word
            possible_word = w_a
            new_word = possible_word
            possible_results.append(new_word)
            print(new_word)

    return possible_results